In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

df = pd.read_csv('train10K.csv')

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

y_train = train_set[["label"]].copy().values.reshape(-1)
X_train = train_set.drop(["approved","amount","label"],axis=1).values.astype(np.float32)

y_test = test_set[["label"]].copy().values.reshape(-1)
X_test = test_set.drop(["approved","amount","label"],axis=1).values.astype(np.float32)

X_valid, X_train = X_train[:1000], X_train[1000:]
y_valid, y_train = y_train[:1000], y_train[1000:]

In [4]:
n_inputs = 8*1
n_hidden1 = 300
n_hidden2 = 150
n_outputs = 11

In [6]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [8]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [9]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [10]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [11]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [15]:
n_epochs = 500
n_batches = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, n_batches):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./cont_model.ckpt")

0 Batch accuracy: 0.46 Validation accuracy: 0.42
1 Batch accuracy: 0.46 Validation accuracy: 0.399
2 Batch accuracy: 0.38 Validation accuracy: 0.407
3 Batch accuracy: 0.42 Validation accuracy: 0.517
4 Batch accuracy: 0.54 Validation accuracy: 0.523
5 Batch accuracy: 0.56 Validation accuracy: 0.531
6 Batch accuracy: 0.4 Validation accuracy: 0.539
7 Batch accuracy: 0.46 Validation accuracy: 0.474
8 Batch accuracy: 0.46 Validation accuracy: 0.459
9 Batch accuracy: 0.64 Validation accuracy: 0.594
10 Batch accuracy: 0.38 Validation accuracy: 0.459
11 Batch accuracy: 0.72 Validation accuracy: 0.593
12 Batch accuracy: 0.48 Validation accuracy: 0.453
13 Batch accuracy: 0.48 Validation accuracy: 0.56
14 Batch accuracy: 0.6 Validation accuracy: 0.541
15 Batch accuracy: 0.5 Validation accuracy: 0.461
16 Batch accuracy: 0.52 Validation accuracy: 0.57
17 Batch accuracy: 0.62 Validation accuracy: 0.596
18 Batch accuracy: 0.64 Validation accuracy: 0.583
19 Batch accuracy: 0.74 Validation accuracy: 0.

In [16]:
final_model_path = "./cont_model.ckpt"
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from ./cont_model.ckpt


In [17]:
accuracy_val

0.93849999